In [0]:
%run "/Workspace/Users/sundaramloganathan@outlook.com/includes/storage_acc_config"

In [0]:
%run "../includes/folder_path"

In [0]:
dbutils.widgets.text("p_owner_name","")
v_owner_name = dbutils.widgets.get("p_owner_name")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import current_timestamp, lit

races_schema = StructType([
    StructField("raceId", IntegerType(), True),
    StructField("year", IntegerType(), False),
    StructField("round", IntegerType(), False),
    StructField("circuitId", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("date", DateType(), False),
    StructField("time", StringType(), False),
    StructField("url", StringType(), False)  
])

In [0]:
races_df = spark.read.schema(races_schema).option("header", "true").csv(f"{bronze_folder_path}/races.csv")

In [0]:
from pyspark.sql.functions import current_timestamp, lit, to_timestamp, concat_ws, col

races_updated_df = races_df.withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("year", "race_year") \
    .withColumnRenamed("circuitId", "circuit_id") \
    .withColumn("race_timestamp", to_timestamp(concat_ws(' ', col('date'), col('time')), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("ingested_by",lit(v_owner_name)) \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("Url", lit(None)) \
    .drop("url") \
    .drop("time") \
    .drop("date")    

In [0]:
# races_updated_df.write.mode("overwrite").parquet("abfss://silver@logandemostorageacc.dfs.core.windows.net/races")
races_updated_df.write.mode("overwrite").partitionBy("race_year").parquet(f"{silver_folder_path}/races")

In [0]:
dbutils.notebook.exit("Success")